In [ ]:
import pandas as pd
from itertools import combinations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_excel('/content/drive/MyDrive/workshop_tutorial/recommendation_system_tutorial/DaxiaoReceipt.xlsx', header=None)

data

,0,1,2,3,4
0,ข้าวหมูกรอบกระเทียม,ข้าวกะเพรากุ้ง,NaN,NaN,NaN
1,ข้าวหมูกรอบคั่วพริกเกลือ,หมูสะเต๊ะ,NaN,NaN,NaN
2,ข้าวปูผัดผงกะหรี่,ข้าวหมูกรอบผัดผงกะหรี่,ข้าวหมูสับกระเทียม,หมูสะเต๊ะ,NaN
3,ข้าวปูผัดผงกะหรี่,ข้าวกะเพราหมูสับ,NaN,NaN,NaN
4,ข้าวปูผัดผงกะหรี่,ข้ามหมูกรอบคั่วพริกเกลือ,NaN,NaN,NaN
5,ข้าวปูผัดผงกะหรี่,ข้าวกะเพราหมูสับ,NaN,NaN,NaN
6,ข้าวปูผัดผงกะหรี่,ข้าวกะเพราหมูสับ,NaN,NaN,NaN
7,ข้าวหมูกรอบกระเทียม,ข้าวหมูกรอบคั่วพริกเกลือ,NaN,NaN,NaN
8,ข้าวหมูกรอบคั่วพริกเกลือ,ข้าวกะเพราหมูสับ,NaN,NaN,NaN
9,ข้าวกะเพราหมูสับ,ข้าวปูคั่วพริกเกลือ,NaN,NaN,NaN


In [ ]:
def find_itemset_support(data, menus):
    itemsets = []
    itemset_support = {}

    i = 0
    itemsets.append([])
    for menu in data.stack().unique():
        temp_set = frozenset([menu])
        itemsets[0].append(temp_set)
    _itemset_support, _garbage_list = calculate_support(data, itemsets[i])
    itemset_support.update(_itemset_support)
    itemsets[i] = del_repeated_itemsets(itemsets[i], _garbage_list)

    isComplete = False
    while not isComplete:
        i = i + 1
        _itemsets = create_itemsets(itemsets[i-1])
        itemsets.append(_itemsets)
        _itemset_support, _garbage_list = calculate_support(data, itemsets[i])
        if len(_itemset_support) == 0:
            isComplete = True
        else:
            itemset_support.update(_itemset_support)
        itemsets[i] = del_repeated_itemsets(itemsets[i], _garbage_list)
        if len(itemsets[i]) == 0:
            del itemsets[i]
    return itemsets, itemset_support

In [ ]:
def calculate_support(data, itemsets_i):
    _itemset_support = {}
    _garbage_list = []
    for i, itemset in enumerate(itemsets_i):
        count_itemset = 0
        for index, transaction in data.iterrows():
            if itemset.issubset(set(transaction)):
                count_itemset = count_itemset + 1
        if count_itemset != 0:
            _itemset_support[itemset] = count_itemset
        else:
            _garbage_list.append(i)
    return _itemset_support, _garbage_list

In [ ]:
def del_repeated_itemsets(itemsets_i, garbage_list):
    for i in range(len(garbage_list)-1, -1, -1):
        del itemsets_i[garbage_list[i]]
    return itemsets_i

In [ ]:
def create_itemsets(itemsets_im1):
    _itemsets = []
    for i in range(len(itemsets_im1)):
        for j in range(i+1, len(itemsets_im1)):
            if len(itemsets_im1[i].intersection(itemsets_im1[j])) == len(itemsets_im1[i]) - 1:
                temp_set = itemsets_im1[i].union(itemsets_im1[j])
            else:
                for item in itemsets_im1[j]:
                    if item not in itemsets_im1[i]:
                        temp_set = itemsets_im1[i].union(set([item]))
            if temp_set not in _itemsets:
                _itemsets.append(temp_set)
    return _itemsets

In [ ]:
def frequent_itemset(itemset_support, minimum_support):
    passed_itemset_support = {}
    for key, value in itemset_support.items():
        if value >= minimum_support:
            passed_itemset_support[key] = value
    return passed_itemset_support

In [ ]:
menus = data.stack().unique()
itemsets, itemset_support = find_itemset_support(data, menus)

In [ ]:
n_transactions = data.shape[0]
minimum_support = n_transactions/10

passed_itemset_support = frequent_itemset(itemset_support, minimum_support)

In [ ]:
minimum_support

2.0

In [ ]:
n_transactions

20

In [ ]:
menus

array(['ข้าวหมูกรอบกระเทียม', 'ข้าวกะเพรากุ้ง',
       'ข้าวหมูกรอบคั่วพริกเกลือ', 'หมูสะเต๊ะ', 'ข้าวปูผัดผงกะหรี่',
       'ข้าวหมูกรอบผัดผงกะหรี่', 'ข้าวหมูสับกระเทียม', 'ข้าวกะเพราหมูสับ',
       'ข้ามหมูกรอบคั่วพริกเกลือ', 'ข้าวปูคั่วพริกเกลือ',
       'ข้าวปูกระเทียม', 'ข้าวกะเพราหมูกรอบ', 'ปูผัดผงกะหรี่',
       'ข้าวไก่กระเทียม', 'ข้าวกะเพราไก่'], dtype=object)

In [ ]:
itemset_support

{frozenset({'ข้าวหมูกรอบกระเทียม'}): 2,
 frozenset({'ข้าวกะเพรากุ้ง'}): 2,
 frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}): 7,
 frozenset({'หมูสะเต๊ะ'}): 5,
 frozenset({'ข้าวปูผัดผงกะหรี่'}): 11,
 frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}): 3,
 frozenset({'ข้าวหมูสับกระเทียม'}): 1,
 frozenset({'ข้าวกะเพราหมูสับ'}): 9,
 frozenset({'ข้ามหมูกรอบคั่วพริกเกลือ'}): 1,
 frozenset({'ข้าวปูคั่วพริกเกลือ'}): 1,
 frozenset({'ข้าวปูกระเทียม'}): 1,
 frozenset({'ข้าวกะเพราหมูกรอบ'}): 1,
 frozenset({'ปูผัดผงกะหรี่'}): 1,
 frozenset({'ข้าวไก่กระเทียม'}): 1,
 frozenset({'ข้าวกะเพราไก่'}): 1,
 frozenset({'ข้าวกะเพรากุ้ง', 'ข้าวหมูกรอบกระเทียม'}): 1,
 frozenset({'ข้าวหมูกรอบกระเทียม', 'ข้าวหมูกรอบคั่วพริกเกลือ'}): 1,
 frozenset({'ข้าวกะเพรากุ้ง', 'ข้าวปูผัดผงกะหรี่'}): 1,
 frozenset({'ข้าวกะเพรากุ้ง', 'ข้าวหมูกรอบผัดผงกะหรี่'}): 1,
 frozenset({'ข้าวกะเพรากุ้ง', 'ข้าวกะเพราหมูสับ'}): 1,
 frozenset({'ข้าวกะเพรากุ้ง', 'ข้าวปูกระเทียม'}): 1,
 frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ', 'หมูสะเต๊ะ'}): 3,
 frozenset({'ข้าวปูผั

In [ ]:
passed_itemset_support

{frozenset({'ข้าวหมูกรอบกระเทียม'}): 2,
 frozenset({'ข้าวกะเพรากุ้ง'}): 2,
 frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}): 7,
 frozenset({'หมูสะเต๊ะ'}): 5,
 frozenset({'ข้าวปูผัดผงกะหรี่'}): 11,
 frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}): 3,
 frozenset({'ข้าวกะเพราหมูสับ'}): 9,
 frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ', 'หมูสะเต๊ะ'}): 3,
 frozenset({'ข้าวปูผัดผงกะหรี่', 'ข้าวหมูกรอบคั่วพริกเกลือ'}): 2,
 frozenset({'ข้าวปูผัดผงกะหรี่', 'ข้าวหมูกรอบผัดผงกะหรี่'}): 3,
 frozenset({'ข้าวกะเพราหมูสับ', 'ข้าวปูผัดผงกะหรี่'}): 5,
 frozenset({'ข้าวกะเพราหมูสับ', 'ข้าวหมูกรอบผัดผงกะหรี่'}): 2,
 frozenset({'ข้าวกะเพราหมูสับ',
            'ข้าวปูผัดผงกะหรี่',
            'ข้าวหมูกรอบผัดผงกะหรี่'}): 2}

In [ ]:
for itemset in passed_itemset_support:
    if len(itemset) >= 2:
        for i in range(len(itemset)-1, len(itemset)):
            all_combinations = list(combinations(itemset, i))
            for comb in all_combinations:
                confidence = passed_itemset_support[itemset]/passed_itemset_support[frozenset(comb)]
                print(f"{frozenset(comb)} ==> {itemset - frozenset(comb)}: {confidence}")
            print()

frozenset({'หมูสะเต๊ะ'}) ==> frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}): 0.6
frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}) ==> frozenset({'หมูสะเต๊ะ'}): 0.42857142857142855

frozenset({'ข้าวปูผัดผงกะหรี่'}) ==> frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}): 0.18181818181818182
frozenset({'ข้าวหมูกรอบคั่วพริกเกลือ'}) ==> frozenset({'ข้าวปูผัดผงกะหรี่'}): 0.2857142857142857

frozenset({'ข้าวปูผัดผงกะหรี่'}) ==> frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}): 0.2727272727272727
frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}) ==> frozenset({'ข้าวปูผัดผงกะหรี่'}): 1.0

frozenset({'ข้าวกะเพราหมูสับ'}) ==> frozenset({'ข้าวปูผัดผงกะหรี่'}): 0.5555555555555556
frozenset({'ข้าวปูผัดผงกะหรี่'}) ==> frozenset({'ข้าวกะเพราหมูสับ'}): 0.45454545454545453

frozenset({'ข้าวกะเพราหมูสับ'}) ==> frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}): 0.2222222222222222
frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'}) ==> frozenset({'ข้าวกะเพราหมูสับ'}): 0.6666666666666666

frozenset({'ข้าวกะเพราหมูสับ', 'ข้าวปูผัดผงกะหรี่'}) ==> frozenset({'ข้าวหมูกรอบผัดผงกะหรี่'})